# Load libraries

In [4]:
import numpy as np
import pandas as pd
import os
import datetime

# Specify git executable file for GitPython in Jupyter Notebook (In IDE, it can still work without this line.)
# os.environ["GIT_PYTHON_GIT_EXECUTABLE"] = "C:\Program Files\Git\cmd\git.exe"

import git
from git import RemoteProgress

from git import Repo
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

# Clone repo from GitHub

Link: https://git-scm.com/docs/git-clone <br>
<b>Note:</b> In case too-long file path issue occurs in Windows, set <code>git config --system core.longpaths true</code>

In [8]:
class Progress(RemoteProgress):
    def update(self, op_code, cur_count, max_count=None, message=''):
        print(self._cur_line)

In [12]:
remote_link = "https://github.com/jenkinsci/jenkins"
local_link = "jenkins"
# Uncomment to clone
#Repo.clone_from(remote_link, local_link, progress=Progress())

In [13]:
repo = Repo(local_link)
fixing_commit = "7b1f8e96a8d97dd09e5e093fcdb010b3295acc77"
affected_file = "core/src/main/java/hudson/model/User.java"

# Task 3

## a. Deleted line

In [83]:
# git blame -L 340,340 User.java
show_data = repo.git.blame("-L340,+1", affected_file).splitlines()
for line in show_data:
    print(line)

7b1f8e96a8d (rysteboe 2017-09-07 16:32:10 -0400 340)             return Collections.unmodifiableList(properties);


## b. Added lines

In [86]:
# git blame -L 340,340 User.java
show_data = repo.git.blame("-L339,+5", affected_file).splitlines()
for line in show_data:
    print(line)

7b1f8e96a8d (rysteboe 2017-09-07 16:32:10 -0400 339)         if (hasPermission(Jenkins.ADMINISTER)) {
7b1f8e96a8d (rysteboe 2017-09-07 16:32:10 -0400 340)             return Collections.unmodifiableList(properties);
7b1f8e96a8d (rysteboe 2017-09-07 16:32:10 -0400 341)         }
7b1f8e96a8d (rysteboe 2017-09-07 16:32:10 -0400 342) 
7b1f8e96a8d (rysteboe 2017-09-07 16:32:10 -0400 343)         return Collections.emptyList();


# Tast 5

## a. Title and message

In [23]:
show_data = repo.git.show("-s", fixing_commit).splitlines()
for line in show_data:
    print(line)

commit 7b1f8e96a8d97dd09e5e093fcdb010b3295acc77
Author: rysteboe <Sicarri@users.noreply.github.com>
Date:   Thu Sep 7 16:32:10 2017 -0400

    [SECURITY-514] Restrict access to user properties via the api to admins


## b. How many total files were affected in the current VCC

In [24]:
show_data = repo.git.diff("--stat", fixing_commit+"^", fixing_commit).splitlines()
for line in show_data:
    print(line)

 core/src/main/java/hudson/model/User.java     |  6 +++++-
 test/src/test/java/hudson/model/UserTest.java | 30 +++++++++++++++++++++++++++
 2 files changed, 35 insertions(+), 1 deletion(-)


## c. How many total directories were affected in the current VCC

In [ ]:
show_data = repo.git.diff("--stat", fixing_commit+"^", fixing_commit).splitlines()
for line in show_data:
    print(line)

## d. How many total lines of code (including comments and blank lines) were deleted

In [77]:
show_data = repo.git.diff("--stat", fixing_commit+"^", fixing_commit).splitlines()
for line in show_data:
    print(line)

 core/src/main/java/hudson/model/User.java     |  6 +++++-
 test/src/test/java/hudson/model/UserTest.java | 30 +++++++++++++++++++++++++++
 2 files changed, 35 insertions(+), 1 deletion(-)


## e. How many total lines of code (including comments and blank lines) were added

In [78]:
show_data = repo.git.diff("--stat", fixing_commit+"^", fixing_commit).splitlines()
for line in show_data:
    print(line)

 core/src/main/java/hudson/model/User.java     |  6 +++++-
 test/src/test/java/hudson/model/UserTest.java | 30 +++++++++++++++++++++++++++
 2 files changed, 35 insertions(+), 1 deletion(-)


## f. How many total lines of code (excluding comments and blank lines) were deleted

In [79]:
show_data = repo.git.diff("--stat",'-w', fixing_commit+"^", fixing_commit).splitlines()
for line in show_data:
    print(line)

 core/src/main/java/hudson/model/User.java     |  4 ++++
 test/src/test/java/hudson/model/UserTest.java | 30 +++++++++++++++++++++++++++
 2 files changed, 34 insertions(+)


## g. How many total lines of code (excluding comments and blank lines) were added

In [80]:
show_data = repo.git.diff("--stat",'-w', fixing_commit+"^", fixing_commit).splitlines()
for line in show_data:
    print(line)

 core/src/main/java/hudson/model/User.java     |  4 ++++
 test/src/test/java/hudson/model/UserTest.java | 30 +++++++++++++++++++++++++++
 2 files changed, 34 insertions(+)


## h. How  many  days  were  between  the  current  VCC  and  the previous commit of each affected file

In [45]:
# git log -1 --pretty=format:"%ad" --date=short 7b1f8e96a8d9
# git log -1 --pretty=format:"%ad" --date=short --before="2017-09-07" UserTest.java
    
VCC_date = repo.git.log('-1','--pretty=format:"%ad"','--date=short', fixing_commit).splitlines()[0].replace('"', "")
previous_commit_date = repo.git.log('-1','--pretty=format:"%ad"','--date=short','--before='+VCC_date, affected_file).splitlines()[0].replace('"', "")
days = datetime.datetime.strptime(VCC_date, '%Y-%m-%d') - datetime.datetime.strptime(previous_commit_date, '%Y-%m-%d')
print(days.days)

5


## i. How  many  times  has  each  affected  file  of  the  current  VCC been  modified  in  the  past  since  their  creation  (including rename of the file)

In [50]:
# git log --oneline --before="2017-09-07" User.java | wc -l

show_data = repo.git.log('--oneline','--before='+VCC_date, affected_file).splitlines()
print(len(show_data))

194


In [54]:
# git log --pretty=format:"%an" --before="2017-09-07" User.java 

show_data = set(repo.git.log('--pretty=format:"%an"','--before='+VCC_date, affected_file).splitlines())
for line in show_data:
    print(line)

"Kohsuke Kawaguchi"
"Baptiste Mathus"
"Kanstantsin Shautsou"
"Oliver Gondža"
"Félix Belzunce Arcos"
"Cyrille Le Clerc"
"abayer"
"Stephen Connolly"
"redsolo"
"Johannes Ernst"
"Jesse Glick"
"Ryan Campbell"
"mindless"
"Nicolas De Loof"
"Robert Sandell"
"Tom Huybrechts"
"huybrechts"
"Daniel Beck"
"Nigel Magnay"
"Seiji Sogabe"
"David Hoover"
"Oleg Nenashev"
"Vincent Latombe"
"christ66"
"kohsuke"
"OHTAKE Tomohiro"
"jglick"
"sogabe"
"James Nord"
"Josh Soref"
"Christoph Kutzinski"


## k. For each developer identified, how many commits have each of them submitted

In [76]:
# git log --pretty=short --before="2017-09-07" User.java | git shortlog -sn 

show_data = repo.git.shortlog('-sn',"HEAD", affected_file).splitlines()
for line in show_data:
    print(line)

    58	Jesse Glick
    57	kohsuke
    26	Daniel Beck
    18	Kohsuke Kawaguchi
    12	Oleg Nenashev
    12	Stephen Connolly
     8	Nicolas De Loof
     8	Wadeck Follonier
     7	mindless
     6	James Nord
     5	Robert Sandell
     3	Josh Soref
     3	Ryan Campbell
     3	huybrechts
     2	Félix Belzunce Arcos
     2	Francisco Fernández
     2	Jeff Thompson
     2	Johannes Ernst
     2	Nigel Magnay
     2	christ66
     1	Baptiste Mathus
     1	Basil Crow
     1	Christoph Kutzinski
     1	Cyrille Le Clerc
     1	David Hoover
     1	Kanstantsin Shautsou
     1	Matt Sicker
     1	OHTAKE Tomohiro
     1	Oliver Gondža
     1	Rebecca Ysteboe
     1	Seiji Sogabe
     1	Stefan Spieker
     1	Tom Huybrechts
     1	Vincent Latombe
     1	abayer
     1	jglick
     1	redsolo
     1	rysteboe
     1	sogabe
